<a href="https://colab.research.google.com/github/y-hayama/notebook/blob/main/ocr_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install pdfminer.six
!pip install python-dotenv

In [ ]:
import dotenv
import os

dotenv.load_dotenv('./env')
print(os.environ['OPENAI_API_KEY'])


In [84]:
from typing import Callable
import time

def time_measure(func: Callable[..., any]) -> Callable[..., any]:
    def wrapper(*args: any, **kwargs: any) -> any:
        before = time.time()
        result = func(*args, **kwargs)
        after = time.time()
        print(f"{func.__name__} proccesing time: {after - before}s")
        return result

    return wrapper

In [85]:
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

@time_measure
def chat_completion(prompt, temperature=0):
  messages=[
    {"role": "user", "content": prompt}
  ]
  result = _chat_completion(messages, temperature)
  # print(result)
  return result.choices[0].message.content

def _chat_completion(messages, temperature=0):
  return openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=temperature,
  )


In [54]:
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter, PDFPageAggregator
from pdfminer.layout import LAParams, LTContainer, LTTextBox
from io import StringIO
import pandas as pd

In [43]:
#v1
def pdf2txt(path):
  with open(path, 'rb') as fp:
    # 出力先をPythonコンソールするためにIOストリームを取得
    with StringIO() as outfp:
      # 各種テキスト抽出に必要なPdfminer.sixのオブジェクトを取得する処理
      rmgr = PDFResourceManager() # PDFResourceManagerオブジェクトの取得
      lprms = LAParams()          # LAParamsオブジェクトの取得
      device = TextConverter(rmgr, outfp, laparams=lprms)    # TextConverterオブジェクトの取得
      iprtr = PDFPageInterpreter(rmgr, device) # PDFPageInterpreterオブジェクトの取得

      # PDFファイルから1ページずつ解析(テキスト抽出)処理する
      for page in PDFPage.get_pages(fp):
        iprtr.process_page(page)

      text = outfp.getvalue()  # Pythonコンソールへの出力内容を取得
      device.close() # TextConverterオブジェクトの解放

  return text

In [86]:
def find_textboxes(layout):
  if isinstance(layout, LTTextBox):
      return [layout]
  elif isinstance(layout, LTContainer):
      boxes = []
      for child in layout:
          boxes.extend(find_textboxes(child))
      return boxes
  else:
      return []

#v2
@time_measure
def pdf2txt(path):
  with open(path, 'rb') as fp:
    # 出力先をPythonコンソールするためにIOストリームを取得
    with StringIO() as outfp:
      # 各種テキスト抽出に必要なPdfminer.sixのオブジェクトを取得する処理
      rmgr = PDFResourceManager() # PDFResourceManagerオブジェクトの取得
      lprms = LAParams()          # LAParamsオブジェクトの取得
      device = PDFPageAggregator(rmgr, laparams=LAParams())
      iprtr = PDFPageInterpreter(rmgr, device) # PDFPageInterpreterオブジェクトの取得
      df = pd.DataFrame()

      # PDFファイルから1ページずつ解析(テキスト抽出)処理する
      for page_no, page in enumerate(PDFPage.get_pages(fp), start=1):
        iprtr.process_page(page)
        #LTPageオブジェクトを取得
        layout = device.get_result()
        #1ページ内のテキストのまとまりのリストを取得
        boxes = find_textboxes(layout)

        #テキストひとまとまりごとに処理
        for box in boxes:
          df_page = pd.DataFrame({
              "x_start":[box.x0],
              "x_end"  :box.x1,
              "y_start":box.y0,
              "y_end"  :box.y1,
              "text"   :box.get_text().strip(),
              "page"   :page_no
            })
          df = pd.concat([df, df_page])

      result = df.reset_index(drop=True)
      device.close() # TextConverterオブジェクトの解放

  return result.to_string()


In [ ]:
text = pdf2txt("/content/pdf/2.pdf")

prompt = """
以下の表は請求書PDFから文字列を抽出した結果です。
このデータは抽出した文字列とページ番号、文字列が含まれていた2つの座標から構成されています。

---
%text%
---

上記の表から請求金額、請求日、支払い期限、銀行名、支店名、口座名、口座番号、適格請求書発行事業者の登録番号とそれに対応する位置情報、ページ番号を以下のjsonフォーマットで取り出してください。
また数字と日付についてはプログラムで処理しやすい形に正規化した結果も返却してください。
与えられたデータ以外は使わず、見つからなかった項目はN/Aを返してください。

フォーマット）
{
    '請求金額': {
        'page': '<ページ番号>',
        'value': '<抽出した値>',
        'normalized_value': '<正規化した値>',
        'poly': '<位置情報>'
    }
}
"""

result = chat_completion(prompt.replace('%text%', text))

print(result)